In [1]:
from __future__ import print_function, division

In [2]:
# This changes the current directory to the base saga directory - make sure to run this first!
# This is necessary to be able to import the py files and use the right directories,
# while keeping all the notebooks in their own directory.
import os
import sys
import time

if 'saga_base_dir' not in locals():
    saga_base_dir = os.path.abspath('..')
if saga_base_dir not in sys.path:
    os.chdir(saga_base_dir)

In [3]:
for module in ['hosts', 'targeting', 'aat']:
    if module in globals():
        reload(globals()[module])
    else:
        globals()[module] = __import__(module)
#g = targeting.get_gama() #re-caches the gama catalog

In [4]:
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import rcParams

rcParams['figure.figsize'] = (16, 10)
rcParams['image.interpolation'] = 'none'
rcParams['image.origin'] = 'lower'

In [5]:
import time
import shutil

from lxml import html
import numpy  as np

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.io import fits
from astropy.utils.data import get_file_contents, download_file
from astropy.table import Table

In [34]:
anaK_coo = SkyCoord(354.13105403*u.deg, 0.29726505 *u.deg, distance=34.4*u.Mpc)

In [7]:
brickurl = 'http://portal.nersc.gov/project/cosmo/data/legacysurvey/dr1/decals-bricks.fits'
bricktab = Table(fits.HDUList.fromstring(get_file_contents(brickurl, cache=True))[1].data)

#re-assign the "has" columns from integers to bools
for cname in bricktab.colnames:
    if cname.startswith('has_'):
        bricktab[cname].dtype = bool

In [8]:
hasall = bricktab['has_image_g']&bricktab['has_image_r']&bricktab['has_image_z']&bricktab['has_catalog']
allbricktab = bricktab[hasall]

In [62]:
point_in_brick = lambda bricktab, ra, dec: (bricktab['ra1']<ra.deg)&(ra.deg<bricktab['ra2'])&(bricktab['dec1']<dec.deg)&(dec.deg<bricktab['dec2'])
def circle_in_brick(bricktab, center, radius):
    
    ras = [[b['ra1'], b['ra1'], b['ra2'], b['ra2']] for b in bricktab]
    decs = [[b['dec1'], b['dec2'], b['dec1'], b['dec2']] for b in bricktab]
    
    
    if radius.unit.is_equivalent(u.deg):
        brickcorners = SkyCoord(ras*u.deg, decs*u.deg)
        brickcornersin = anaK_coo.separation(brickcorners) < radius
    elif radius.unit.is_equivalent(u.kpc):
        brickcorners = SkyCoord(ras*u.deg, decs*u.deg, distance=center.distance)
        brickcornersin = anaK_coo.separation_3d(brickcorners) < radius
        #radius = ((radius/center.distance)*u.rad).to(u.deg)
    else:
        raise ValueError('invalid unit on radius')
        
    return np.any(brickcornersin, axis=1)

In [64]:
anakonlyin = inbrick(allbricktab, anaK_coo.ra, anaK_coo.dec)
anakin = circle_in_brick(allbricktab, anaK_coo, 400*u.kpc)
np.sum(anakonlyin), np.sum(anakin)

(1, 34)

In [65]:
basecaturl = 'http://portal.nersc.gov/project/cosmo/data/legacysurvey/dr1/tractor/'
baseimurl = 'http://portal.nersc.gov/project/cosmo/data/legacysurvey/dr1/coadd/'

caturls = []
imdirsurls = []
for entry in allbricktab[anakin]:
    brickname = entry['brickname']
    dirname = brickname[:3]
    caturls.append(basecaturl + '{dirname}/tractor-{brickname}.fits'.format(**locals()))
    imdirsurls.append(baseimurl + '{dirname}/{brickname}/'.format(**locals()))

In [67]:
#now actually download the catalogs
cattabs = [Table(fits.HDUList.fromstring(get_file_contents(url, cache=True))[1].data) for url in caturls]

In [87]:
#download the images to a local dir because they are quite big
todl = []
for imdir in imdirsurls:
    tree = html.parse(imdir)
    hrefs = [a.attrib['href'] for a in tree.findall(".//a")]
    #todl.extend([imdir+href for href in hrefs if '-image' in href or '-model' in href or '-resid' in href])
    todl.extend([imdir+href for href in hrefs if '-image-r.fits' in href or 
                                                 '-image.jpg' in href or 
                                                 '-model-r.fits.gz' in href or
                                                 '-model.jpg' in href])
print([i.split('/')[-1] for i in todl])

['decals-3536m002-image-r.fits', 'decals-3536m002-image.jpg', 'decals-3536m002-model-r.fits.gz', 'decals-3536m002-model.jpg', 'decals-3538m002-image-r.fits', 'decals-3538m002-image.jpg', 'decals-3538m002-model-r.fits.gz', 'decals-3538m002-model.jpg', 'decals-3541m002-image-r.fits', 'decals-3541m002-image.jpg', 'decals-3541m002-model-r.fits.gz', 'decals-3541m002-model.jpg', 'decals-3543m002-image-r.fits', 'decals-3543m002-image.jpg', 'decals-3543m002-model-r.fits.gz', 'decals-3543m002-model.jpg', 'decals-3546m002-image-r.fits', 'decals-3546m002-image.jpg', 'decals-3546m002-model-r.fits.gz', 'decals-3546m002-model.jpg', 'decals-3533p000-image-r.fits', 'decals-3533p000-image.jpg', 'decals-3533p000-model-r.fits.gz', 'decals-3533p000-model.jpg', 'decals-3536p000-image-r.fits', 'decals-3536p000-image.jpg', 'decals-3536p000-model-r.fits.gz', 'decals-3536p000-model.jpg', 'decals-3538p000-image-r.fits', 'decals-3538p000-image.jpg', 'decals-3538p000-model-r.fits.gz', 'decals-3538p000-model.jpg',

In [88]:
dlednames = []
for i, url in enumerate(todl):
    print('On file', i+1, 'of', len(todl))
    targetname = os.path.join('decals_images', url.split('/')[-1])
    dlednames.append(targetname)
    if os.path.isfile(targetname):
        print('File', targetname, 'exists, skipping')
        continue
    st = time.time()
    dlname = download_file(url, cache=False)
    print('Took', time.time()-st, 'sec. Moving', dlname, 'to', targetname)
    shutil.move(dlname, targetname)

On file 1 of 136
File decals_images/decals-3536m002-image-r.fits exists, skipping
On file 2 of 136
File decals_images/decals-3536m002-image.jpg exists, skipping
On file 3 of 136
Took 52.4696760178 sec. Moving /var/folders/7_/0n3gbrls1sb2vn6xjprw8c280000gn/T/tmpe7IMyb to decals_images/decals-3536m002-model-r.fits.gz
On file 4 of 136
Took 1.33755207062 sec. Moving /var/folders/7_/0n3gbrls1sb2vn6xjprw8c280000gn/T/tmpkMxAV5 to decals_images/decals-3536m002-model.jpg
On file 5 of 136
File decals_images/decals-3538m002-image-r.fits exists, skipping
On file 6 of 136
File decals_images/decals-3538m002-image.jpg exists, skipping
On file 7 of 136
Took 64.2111768723 sec. Moving /var/folders/7_/0n3gbrls1sb2vn6xjprw8c280000gn/T/tmpsJDxho to decals_images/decals-3538m002-model-r.fits.gz
On file 8 of 136
Took 1.52127194405 sec. Moving /var/folders/7_/0n3gbrls1sb2vn6xjprw8c280000gn/T/tmptEd3sI to decals_images/decals-3538m002-model.jpg
On file 9 of 136
File decals_images/decals-3541m002-image-r.fits e

In [30]:
#build residual maps for all image/model sets
for fn in dlednames:
    if '-image-' in fn:
        modelfn = fn.replace('-image-', '-model-').replace('.fits', '.fits.gz')
        if modelfn not in dlednames:
            print('Found', fn, 'but no model image', modelfn)
        else:
            residfn = fn.replace('-image-', '-resid-')
            if os.path.isfile(residfn):
                print('Residual file', residfn, 'already exists.  Skipping.')
                continue
            ifn = fits.open(fn)
            modeldata = fits.getdata(modelfn)
            resid = ifn[0].data - modeldata
            ifn[0].data = resid
            print('Writing residual file', residfn)
            ifn.writeto(residfn)

Writing residual file decals_images/decals-3541p002-resid-g.fits
Writing residual file decals_images/decals-3541p002-resid-r.fits
Writing residual file decals_images/decals-3541p002-resid-z.fits
